## Convocatoria 1 - Proyecto 2

In [1]:
## Cargamos todas las librerias necesarias
import numpy as np
from tabulate import tabulate
import matplotlib.pyplot as plt
import pandas as pd

# procesado de imagenes
import cv2
from skimage import io
from skimage import img_as_ubyte # skimage image to opencv
from skimage import morphology
from skimage import util #invertir imagen
from skimage.measure import label
from skimage.measure import regionprops
from skimage.morphology import convex_hull_image
import scipy

# outliers
from scipy import stats
from sklearn.covariance import EllipticEnvelope
from limpieza_funciones import grafico_outliers

# para crear clasificador
import math
from math import sqrt
from sklearn import datasets, linear_model
from sklearn import preprocessing
import sklearn.metrics as metrics
from sklearn.metrics import make_scorer, mean_squared_error

from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import cross_val_predict, cross_val_score, train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import SelectPercentile, f_regression
from evaluacion_funciones import *
from pprint import pprint

### Ejercicio 1 (4 puntos)
Utilizar el conjunto de datos "dataset_1.npy" para resolver el ejercicio. Tener en cuenta que la última columna corresponde a la clase.

In [2]:
# Cargamos el dataset
data = np.load('dataset_1.npy')
data_df = pd.DataFrame(data) # lo pasamos a dataframe por si acaso, aunque no hace falta 

# lo guardo como csv para poder interpretarlo mas facilmente
data_df.to_csv('dataset_1.csv')

# display(len(data)) #500x9
display(data_df.head(5))
display(data_df[13:14]) # represento esta por que se que tiene Nan's

,0,1,2,3,4,5,6,7,8
0,0.0,4.0,1.0,4.0,2.0,9.044810,8.260363,7.374902,0.0
1,0.0,4.0,0.0,6.0,1.0,8.569082,10.937074,10.301310,0.0
2,0.0,3.0,1.0,1.0,5.0,9.687778,10.691348,11.456714,0.0
3,0.0,2.0,1.0,5.0,1.0,12.057221,10.286357,10.575028,0.0
4,0.0,2.0,2.0,3.0,2.0,9.732969,9.447140,9.383029,0.0


,0,1,2,3,4,5,6,7,8
13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


In [127]:
display(data_df.sample(10))

,0,1,2,3,4,5,6,7,8
12,0.0,3.0,3.0,2.0,2.0,6.444211,7.755673,7.038937,0.0
64,0.0,1.0,1.0,6.0,0.0,9.929024,9.066996,9.386130,0.0
447,0.0,4.0,1.0,4.0,2.0,2.002706,6.107761,1.104373,0.0
352,0.0,3.0,2.0,5.0,0.0,NaN,13.752695,10.000000,1.0
198,0.0,1.0,2.0,3.0,2.0,9.573755,9.396510,8.291987,0.0
328,0.0,2.0,3.0,2.0,2.0,13.464732,10.972380,10.000000,1.0
28,0.0,3.0,2.0,3.0,2.0,10.427293,8.221522,7.791552,1.0
471,0.0,1.0,0.0,5.0,2.0,2.902839,2.176934,8.934040,0.0
204,0.0,3.0,1.0,5.0,1.0,11.419950,10.573331,11.293825,0.0
62,0.0,2.0,2.0,4.0,1.0,9.614163,10.757391,10.581931,0.0


In [3]:
# imprimimos la informacion de nulos del dataset

def valores_nulos(df):
    if(df.isnull().values.any()): # Comprobamos si hay valores NaN
        print('\nnº de valores nulos por columna:')
        print(df.isna().sum()) # si los hay, ver cuantos
        print('\nnº de valores nulos por fila:')
        print(df.shape[1] - df.count(axis=1)) # Número de nulos por fila
    else:
        print('no hay valores nulos')
    return 

#### 1.a) Imputación de valores ausentes (2 puntos)

- En aquellas instancias (filas) que contengan UN ÚNICO VALOR NaN en alguno de sus atributos (columnas), se imputará dicho valor.
     * Si el atributo corresponde a una variable discreta, se imputará el valor utilizando la moda de dicho atributo. 
     * Si el atributo corresponde a una variable continua, se imputará el valor utilizando la media de dicho atributo. 
- Aquellas instancias (filas) que contengan MÁS DE UN VALOR NaN en sus atributos, deberán ser eliminadas por completo. 

Los outputs deberán ser: 
- Una matriz "X" de dimensiones M x N, donde M será el número de instancias y N, el de atributos.
- Un vector "y" de dimensiones M. 

In [4]:
data_nan = data_df.copy()

# las filas con 2 o mas Nan se eliminan por completo
data_nan = data_nan.dropna(thresh=2, axis=0, inplace=False) # elimina si hay mas de 2 por fila

# las filas que no tengan  mas de dos valores Nan, se sustituye por la moda o la media
data_nan = data_nan.fillna(-1) # sustituir por un valor fijo (de momento)

# data_nan.info() # info del dataset
valores_nulos(data_nan)
# display(data_nan[12:14])
# display(data_nan[352:353])

no hay valores nulos


In [7]:
# Dividir dataset en matriz X con atributos, y vector Y con clases

data_x = data_nan.copy().drop([8],axis=1) 
data_y = data_nan[8].copy()

display(data_x.head())
display(data_y.head())

AttributeError: 'numpy.ndarray' object has no attribute 'target'

#### 1.b) Eliminación de valores outliers extremos (2 puntos)
- Aplicar el método de Elliptic Envelope fijando una semilla en 42
- Utilizar el diagrama de caja y bigotes para establecer los umbrales de decisión
- Plotear el diagrama (box and whiskers) obtenido a partir de las puntuaciones calculadas con el método Elliptic Envelope
- Eliminar las instancias que contengan valores outliers extremos

Los outputs deberán ser: 
- Una matriz "X" de dimensiones M' x N, donde M' será el nuevo número de instancias y N, el de atributos.
- Un vector "y" de dimensiones M'. 

In [145]:
# Limpieza de datos: detección de outliers.
outlier_method = EllipticEnvelope().fit(data_x)
scores_pred = outlier_method.decision_function(data_x)
threshold = stats.scoreatpercentile(scores_pred, 25)

E:\Programs\conda\lib\site-packages\sklearn\covariance\_robust_covariance.py:166: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-35.472558227105260 > -39.679733627911546). You may want to try with a higher value of support_fraction (current value: 0.509).
  warnings.warn("Determinant has increased; this should not happen: "
E:\Programs\conda\lib\site-packages\sklearn\covariance\_robust_covariance.py:166: RuntimeWarning: Determinant has increased; this should not happen: log(det) > log(previous_det) (-35.874216857519222 > -38.027595934221559). You may want to try with a higher value of support_fraction (current value: 0.509).
  warnings.warn("Determinant has increased; this should not happen: "


In [141]:
# Ratio de outliers.
print("%.3f %%" % (100*len(scores_pred[scores_pred < threshold])/len(scores_pred)))

25.051 %


In [147]:
# Dibujar gráfica de outliers.
grafico_outliers(data_x, outlier_method, 150, threshold, -7, 7) # no funciona, hacerlo para cada atributo por separado?

ValueError: Incompatible dimension for X and Y matrices: X.shape[1] == 2 while Y.shape[1] == 8

### Ejercicio 2 (6 puntos)

Crear un programa, con comentarios que aclaren el código, donde se computen las métricas MAE, MSE, RMSE, MAPE y R2 aplicando los regresores OLS y KNN en DOS datasets de regresión (a elegir). Nota: Al menos uno de los datasets deberá ser externo a la librería de scikit-learn.

Se debe llevar a cabo:
- Exploración de los datos y descripción del dataset
- Tratamiento de outliers y valores perdidos (si los hay)
- Partición externa de datos
- Estandarización de los datos
- Selección de atributos mediante métodos supervisados (solo uno)
- Validación cruzada interna para optimizar los hiperparámetros de los algoritmos (si procede)
- Modelado
- Predicción y evaluación sobre el conjunto de test (resultados cuantitativos y cualitativos)
- Breve discusión comparando los resultados obtenidos con cada método.

In [ ]:
# Métricas de evaluación.
metricas = {
    'MAE': 'neg_mean_absolute_error',
    'MSE': 'neg_mean_squared_error',
    'RMSE': 'neg_root_mean_squared_error',
    'MAPE': lambda y, y_pred: np.mean(np.abs((y - y_pred) / y)) * 100,
    'R2':   'r2'
}

In [ ]:
# encontrar DOS datasets de regresion
# aplicar regresores OLS y KNN
# obtener metricas

In [ ]:
# Cálculo de las métricas de evaluación.
MAE = metrics.mean_absolute_error(y_testing, y_pred_test)
MSE = metrics.mean_squared_error(y_testing, y_pred_test, squared=True)
RMSE = metrics.mean_squared_error(y_testing, y_pred_test, squared=False)
MAPE = metrics.mean_squared_error(y_testing, y_pred_test)
R2 = metrics.r2_score(y_testing, y_pred_test)

print('MAE:  %.4f' % MAE)
print('MSE:  %.4f' % MSE)
print('RMSE: %.4f' % RMSE)
print('MAPE: %.4f' % MAPE)
print('R2:   %.4f' % R2)

In [ ]:
# Gráfica de realidad vs. predicción.
fig, ax = plt.subplots()
ax.scatter(y_testing, y_pred_test, edgecolors=(0, 0, 0))
ax.plot([y.min(), y.max()], [y.min(), y.max()], 'k--', lw=4)
ax.set_xlabel('Valor real de la clase')
ax.set_ylabel('Predicción')
plt.title("MAE: %.3f  MSE: %.3f   RMSE: %.3f  MAPE: %.3f  R2: %.3f" %
          (MAE, MSE, RMSE, MAPE, R2))
plt.grid()
plt.show()